In [1]:
from flyvis.datasets.moving_bar import MovingEdge
import numpy as np
import os
from flyvis_cell_type_pert import FlyvisCellTypePert
import pandas as pd
from tqdm import tqdm
from moving_edge import MovingEdgeWrapper
from pathlib import Path
from flyvis_cell_type_pert import PerturbationType

c:\Users\madis\Documents\Perturbations\fly_wire_perturbations\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


 -> Patched datamate.directory._write_h5
Importing flyvis...


In [2]:
import datamate

data_path = Path("data/flyvis_data")
data_path.mkdir(parents=True, exist_ok=True)

env = os.environ.copy()
env["FLYVIS_ROOT_DIR"] = str(data_path)


In [3]:
cell_type_df = pd.read_csv(f'{data_path}/flyvis_cell_type_connectivity.csv')

dataset = MovingEdge(
            offsets=[-10, 11],  # offset of bar from center in 1 * radians(2.25) led size
            intensities=[0, 1],  # intensity of bar
            speeds=[19],  # speed of bar in 1 * radians(5.8) / s
            height=80,  # height of moving bar in 1 * radians(2.25) led size
            post_pad_mode="continue",  # for post-stimulus period, continue with the last frame of the stimulus
            t_pre=1.0,  # duration of pre-stimulus period
            t_post=1.0,  # duration of post-stimulus period
            dt=1 / 200,  # temporal resolution of rendered video
            angles=list(np.arange(0, 360, 30)),  # motion direction (orthogonal to edge)
    )

In [5]:
pairwise_hits = ['CT1(Lo1)_T5b', 'CT1(M10)_Mi1', 'Mi4_T4d', 'R8_Mi4', 'Tm9_T5c', 'TmY15_Tm4', 'Mi1_T4a', 'R8_Mi1', 'Mi4_T4d', 'Mi4_Tm9']

In [6]:
for hit in pairwise_hits:
    src, tar = hit.split('_')
    result_output_path = f'data/flyvis_data/pairwise_edge_pert_results/{src}_{tar}.csv'
    plot_output_dir = f'data/flyvis_data/perf/moving_edge_fig/{src}_{tar}'
    #if os.path.exists(result_output_path):
    #    print(f"Skipping existing perturbation: {src} -> {tar}")
    #    continue
    print(f"Running perturbation: {src} -> {tar}")

    pert = FlyvisCellTypePert()
    pert.perturb(cell_type_df, PerturbationType.PAIR_WISE, pairs=[(src, tar)])

    wrapper = MovingEdgeWrapper(dataset, pert=pert,
                                pert_folder_name=f'{src}_{tar}_perturbation',
                                output_file_name=result_output_path,
                                plot_output_dir=plot_output_dir)
    wrapper.run()

[2026-01-07 20:26:37] network_view:122 Initialized network view at C:\Users\madis\Documents\Perturbations\fly_wire_perturbations\data\flyvis_data\results\flow\0000\000_CT1(Lo1)_T5b_perturbation
[2026-01-07 20:26:37] logging_utils:23 epe not in C:\Users\madis\Documents\Perturbations\fly_wire_perturbations\data\flyvis_data\results\flow\0000\000_CT1(Lo1)_T5b_perturbation\validation, but 'loss' is. Falling back to 'loss'. You can rerun the ensemble validation to make appropriate recordings of the losses.


Running perturbation: CT1(Lo1) -> T5b


[2026-01-07 20:26:41] network:222 Initialized network with NumberOfParams(free=734, fixed=2959) parameters.
[2026-01-07 20:26:41] chkpt_utils:36 Recovered network state.


Running moving edge simulation...
Applying perturbation to network in memory...
Overwriting disk checkpoints with perturbed weights...


[2026-01-07 20:26:41] network_view:122 Initialized network view at C:\Users\madis\Documents\Perturbations\fly_wire_perturbations\data\flyvis_data\results\flow\0000\000_CT1(Lo1)_T5b_perturbation


...

Running moving edge simulation...
Applying perturbation to network in memory...
Overwriting disk checkpoints with perturbed weights...
 -> Updated: data\flyvis_data\results\flow\0000\000_Mi4_Tm9_perturbation\best_chkpt
 -> Updated: data\flyvis_data\results\flow\0000\000_Mi4_Tm9_perturbation\chkpts\chkpt_00000
Clearing caches...


[2026-01-07 20:37:14] network_view:122 Initialized network view at C:\Users\madis\Documents\Perturbations\fly_wire_perturbations\data\flyvis_data\results\flow\0000\000_Mi4_Tm9_perturbation


 -> Removed __cache__
Generating moving edge responses...


[2026-01-07 20:37:18] network:222 Initialized network with NumberOfParams(free=734, fixed=2959) parameters.
[2026-01-07 20:37:18] chkpt_utils:36 Recovered network state.
[2026-01-07 20:37:20] network:757 Computing 24 stimulus responses.
[2026-01-07 20:38:11] xarray_joblib_backend:56 Store item C:\Users\madis\Documents\Perturbations\fly_wire_perturbations\data\flyvis_data\results\flow\0000\000_Mi4_Tm9_perturbation\__cache__\flyvis\analysis\stimulus_responses\compute_responses\4673075b2ebc093e6a4b9058f554a835\output.h5


Evaluating performance...
Generating response plots...
Saved plot: data/flyvis_data/perf/moving_edge_fig/Mi4_Tm9\T4a_intensity0.png
Saved tuning curve: data/flyvis_data/perf/moving_edge_fig/Mi4_Tm9\T4a_tuning_intensity0.png
Saved plot: data/flyvis_data/perf/moving_edge_fig/Mi4_Tm9\T4a_intensity1.png
Saved tuning curve: data/flyvis_data/perf/moving_edge_fig/Mi4_Tm9\T4a_tuning_intensity1.png
Saved plot: data/flyvis_data/perf/moving_edge_fig/Mi4_Tm9\T4b_intensity0.png
Saved tuning curve: data/flyvis_data/perf/moving_edge_fig/Mi4_Tm9\T4b_tuning_intensity0.png
Saved plot: data/flyvis_data/perf/moving_edge_fig/Mi4_Tm9\T4b_intensity1.png
Saved tuning curve: data/flyvis_data/perf/moving_edge_fig/Mi4_Tm9\T4b_tuning_intensity1.png
Saved plot: data/flyvis_data/perf/moving_edge_fig/Mi4_Tm9\T4c_intensity0.png
Saved tuning curve: data/flyvis_data/perf/moving_edge_fig/Mi4_Tm9\T4c_tuning_intensity0.png
Saved plot: data/flyvis_data/perf/moving_edge_fig/Mi4_Tm9\T4c_intensity1.png
Saved tuning curve: dat